In [1]:
import sys
!{sys.executable} -m pip install apyori
import warnings
warnings.filterwarnings('ignore')

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!{sys.executable} -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import sys
!{sys.executable} -m pip install "numpy<2"

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import numpy as np
from apyori import apriori
df = pd.read_csv('Market_Basket_Optimisation.csv',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_filled = df.fillna("0")

In [6]:
N = len(df_filled)


In [7]:
df_filled

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,chutney,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,turkey,avocado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,mineral water,milk,energy bar,whole wheat rice,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7498,chicken,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7499,escalope,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
transactions = []
for i in range(0, len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0, 20)
                         if str(df.values[i,j]) != '0' and str(df.values[i,j]) != 'nan'])

print(f"Số lượng giao dịch: {len(transactions)}")
print(f"Mẫu giao dịch đầu tiên: {transactions[0]}")

Số lượng giao dịch: 7501
Mẫu giao dịch đầu tiên: ['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil']


In [9]:
from apyori import apriori

rules_generator = apriori(transactions, min_support=0.01, min_confidence=0.2, min_lift=2, min_length=2)
rules = list(rules_generator)

print(f"Đã tìm thấy {len(rules)} luật kết hợp.")

Đã tìm thấy 18 luật kết hợp.


In [10]:
import pandas as pd

def inspect(results):
    lhs         = []
    rhs         = []
    supports    = []
    confidences = []
    lifts       = []

    for result in results:
        support = result[1]
        ordered_statistics = result[2]
        for record in ordered_statistics:
            lhs_val = tuple(record[0])
            rhs_val = tuple(record[1])

            lhs.append(lhs_val)
            rhs.append(rhs_val)
            supports.append(support)
            confidences.append(record[2])
            lifts.append(record[3])

    return list(zip(lhs, rhs, supports, confidences, lifts))

result_df = pd.DataFrame(inspect(rules), columns=['antecedents', 'consequents', 'support', 'confidence', 'lift'])

result_df = result_df.sort_values(by="lift", ascending=False)

print(result_df.head(10))

                           antecedents           consequents   support  \
3                     (herb & pepper,)        (ground beef,)  0.015998   
20          (spaghetti, mineral water)        (ground beef,)  0.017064   
1                          (tomatoes,)  (frozen vegetables,)  0.016131   
0                            (shrimp,)  (frozen vegetables,)  0.016664   
14               (mineral water, milk)  (frozen vegetables,)  0.011065   
19        (mineral water, ground beef)          (spaghetti,)  0.017064   
13  (mineral water, frozen vegetables)               (milk,)  0.011065   
18               (mineral water, milk)        (ground beef,)  0.011065   
7                              (soup,)               (milk,)  0.015198   
6                         (spaghetti,)        (ground beef,)  0.039195   

    confidence      lift  
3     0.323450  3.291994  
20    0.285714  2.907928  
1     0.235867  2.474464  
0     0.233209  2.446574  
14    0.230556  2.418737  
19    0.416938  2.39468